# Criando um exemplo real: Syngenta - sementes NK

## 1. Importando as bibliotecas e abrindo o link com todas as sementes

In [83]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [100]:
url = 'http://www.portalsyngenta.com.br/sementes/nk-soja'

html = urlopen(url)
scrap = BeautifulSoup(html, 'html.parser')

## 2. explorado a página html, podemos ver que os 6 cards tem a mesma estrutura:
###  < div class = card-text-2
#### buscando com `find_all`
* Usando uma variavel `class_card` para facilitar o reuso do código

In [ ]:
class_card = "card-text-2 card-portfolio"

cards = scrap.find_all('div', {'class': class_card})
cards

## 3. Varremos os cards com um `for`, para obtermos os links de cada semente NK:
####  < a [ ' href ' ]  ----> armazenando em uma lista
## 4. Juntando o site principal com os links armazenados na lista
#### Os links fornecidos são complementares ao do site principal

In [102]:
links = []
principal = 'http://www.portalsyngenta.com.br'

for link in cards:
    link = principal+(link.a['href'])
    links.append(link)
    
links

['http://www.portalsyngenta.com.br/sementes/nk-soja/nk-8770-ipro',
 'http://www.portalsyngenta.com.br/sementes/nk-soja/nk-7777-ipro',
 'http://www.portalsyngenta.com.br/sementes/nk-soja/nk-8448-ipro',
 'http://www.portalsyngenta.com.br/sementes/nk-soja/nk-8301-ipro',
 'http://www.portalsyngenta.com.br/sementes/nk-soja/nk-7201-ipro',
 'http://www.portalsyngenta.com.br/sementes/nk-soja/nk-6201-ipro']

## 5. Explorando cada link e coletando as informações:

* inserindo variáveis para facilitar o aproveitamento do código:
  * conteúdos dentro de <div <class
  * `class_titulo`
  * `class_descricao`
               
                                    
* guardando as informações em listas                                   

In [104]:
class_titulo = "prop-nk-titulo"
class_descricao = "prop-nk-descricao"
tamanho_nome = -12    # no final do link, as últimas 12 letras correspondem ao nome

lista_titulos = []
lista_descricao = []
lista_nome = []


for link in links:
    url = requests.get(link)
    conteudo = url.content
    scrap = BeautifulSoup(conteudo, 'html.parser')

    titulo = scrap.find_all('div', {'class': class_titulo})
    descricao = scrap.find_all('div', {'class': class_descricao})
    
    lista_nome.append(link[tamanho_nome:])
    
    for tit in titulo:
        lista_titulos.append(tit.p.text)
        
    for desc in descricao:
        lista_descricao.append(desc.p.text)    
        
print(lista_nome)
print(len(lista_descricao))

['nk-8770-ipro', 'nk-7777-ipro', 'nk-8448-ipro', 'nk-8301-ipro', 'nk-7201-ipro', 'nk-6201-ipro']
54


## 6. Criando um array com 6 listas de 9 elementos cada; e inserindo o nome da semente

In [118]:
array = np.array(lista_descricao).reshape(6, 9)

df_nk = pd.DataFrame(data=array)
df_nk['Nome'] = lista_nome
df_nk

,0,1,2,3,4,5,6,7,8,nome
0,semi-determinado,ROXA,8.7,CINZA,Tolerante,Marrom Claro,97 - 112,140 - 190 KG,14 - 19,nk-8770-ipro
1,INDETERMINADO,ROXA,7.7,CINZA,Tolerante,Marrom Claro,80 - 95,190 G,15 - 20,nk-7777-ipro
2,INDETERMINADO,ROXA,8.4,CINZA,Tolerante,Marrom Claro,95 - 105,758 - 797 KG,10 - 45,nk-8448-ipro
3,INDETERMINADO,Branca,8.4,Marrom Clara,Tolerante,Marrom Médio,80 - 95,191 G,10 - 15,nk-8301-ipro
4,Indeterminado,Branca,7.2,CINZA,Resistente,Marrom Claro,80 - 100,187 G,13CM,nk-7201-ipro
5,Indeterminado,Branca,6.2,CINZA,Tolerante,Marrom Claro,67 - 103,174 G,11 - 24,nk-6201-ipro


## 7. Inserindo os nomes das colunas
* a lista_titulos se repente de 9 em 9 elementos
* pegando os elementos únicos e adicionando 'Nome'
* colocando como header a lista contendo 10 elementos com os nomes das colunas

In [119]:
lista_titulos = lista_titulos[:9].copy()
lista_titulos.append('Nome')
df_nk.columns = lista_titulos
df_nk

,Hábito de Crescimento,Cor da Flor,Grupo de Maturação,Pubescência,Acamamento,Hilo,Altura da Planta (cm),"PMS Médio 5,5 mm",Inserção de Vagem (cm),Nome
0,semi-determinado,ROXA,8.7,CINZA,Tolerante,Marrom Claro,97 - 112,140 - 190 KG,14 - 19,nk-8770-ipro
1,INDETERMINADO,ROXA,7.7,CINZA,Tolerante,Marrom Claro,80 - 95,190 G,15 - 20,nk-7777-ipro
2,INDETERMINADO,ROXA,8.4,CINZA,Tolerante,Marrom Claro,95 - 105,758 - 797 KG,10 - 45,nk-8448-ipro
3,INDETERMINADO,Branca,8.4,Marrom Clara,Tolerante,Marrom Médio,80 - 95,191 G,10 - 15,nk-8301-ipro
4,Indeterminado,Branca,7.2,CINZA,Resistente,Marrom Claro,80 - 100,187 G,13CM,nk-7201-ipro
5,Indeterminado,Branca,6.2,CINZA,Tolerante,Marrom Claro,67 - 103,174 G,11 - 24,nk-6201-ipro


## 8. Exportando para arquivo excel, dentro da pasta: planilhas/

In [103]:
df_nk.to_excel('planilhas/Syngenta_sementes_NK.xlsx')